## USDHKD volatility surface build

Given market inputs:

- Spot: $S_0$
- Forward: F
- ATM Call price at strike K = F : $C_{\text{mkt}}$
- ATM Put price at strike K = F : $P_{\text{mkt}}$

We want to solve for jump model parameters:

- $p$: probability of no jump
- $q$: probability of jump down -d 
- $d$: log-jump size (assume symmetric, so +d and -d only

## Model Assumptions

- $U \sim \mathrm{Unif}[a, b]$, e.g. $[7.75, 7.85]$
- Spot return:  
  $$
  S_T = e^{R_{\text{jump}}} \cdot U
  $$
- Three-point jump:
  $$
  R_{\text{jump}} =
  \begin{cases}
  -d & \text{with prob } q \\
  0  & \text{with prob } p \\
  +d & \text{with prob } 1 - p - q
  \end{cases}
  $$

## Step 1: Forward Constraint

From risk-neutral pricing:

$$
F = \mathbb{E}[S_T] = \mu \cdot (q e^{-d} + p + (1 - p - q)e^{d})
\quad \text{where } \mu = \frac{a + b}{2}
\tag{1}
$$

## Step 2: ATM Call Pricing

$$
C_{\text{mkt}} =
q \cdot \Phi_{\text{call}}(F; -d)
+ p \cdot \Phi_{\text{call}}(F; 0)
+ (1 - p - q) \cdot \Phi_{\text{call}}(F; d)
\tag{2}
$$

## Step 3: ATM Put Pricing

$$
P_{\text{mkt}} =
q \cdot \Phi_{\text{put}}(F; -d)
+ p \cdot \Phi_{\text{put}}(F; 0)
+ (1 - p - q) \cdot \Phi_{\text{put}}(F; d)
\tag{3}
$$

## Definitions of $\Phi_{\text{call}}(K;x)$ and $\Phi_{\text{put}}(K;x)$

Let $a_x = a e^{x}$, $b_x = b e^{x}$, $\Delta_x = b_x - a_x = \Delta e^{x}$:

$$
\Phi_{\text{call}}(K;x) =
\begin{cases}
\frac{a_x + b_x}{2} - K, & K \le a_x \\[6pt]
\frac{(b_x - K)^2}{2 \Delta_x}, & a_x < K < b_x \\[6pt]
0, & K \ge b_x
\end{cases}
$$

$$
\Phi_{\text{put}}(K;x) =
\begin{cases}
0, & K \le a_x \\[6pt]
\frac{(K - a_x)^2}{2 \Delta_x}, & a_x < K < b_x \\[6pt]
K - \frac{a_x + b_x}{2}, & K \ge b_x
\end{cases}
$$

## 🚀 Calibration Strategy

### Option A: Fix $p = 0$

Then:

- From (1):
  $$
  q = \frac{e^{d} - \frac{F}{\mu}}{e^{d} - e^{-d}}
  \tag{4}
  $$

- Plug into (2), (3) → only unknown is $d$

Use root solver (Brent / Newton) to solve for $d$

### Option B: General case (solve $p$, $q$, $d$)

1. Use (1) to eliminate $p$:
   $$
   p = \frac{F}{\mu} - q e^{-d} - (1 - p - q)e^{d}
   $$

2. Plug into (2), (3) → Two equations, two unknowns: $q$, $d$

Use root finder / system solver (e.g. `scipy.optimize.root`)

## Final Output

Once you solve for $(p, q, d)$, you can:

- Build full risk-neutral PDF
- Price any vanilla option with closed-form
- Generate implied vol surface
- Derive 25Δ RR, Fly, and Greeks

## 4. Case-by-Case Pricing Formula (7 Cases)

Let $a_{-d} = a e^{-d}$, $b_{-d} = b e^{-d}$, $a_0 = a$, $b_0 = b$, $a_d = a e^d$, $b_d = b e^d$

### Case 1: $K < a e^{-d}$

$$
C(K) = q \left( \frac{a_{-d} + b_{-d}}{2} - K \right)
+ p \left( \frac{a_0 + b_0}{2} - K \right)
+ (1 - p - q) \left( \frac{a_d + b_d}{2} - K \right)
$$

### Case 2: $a e^{-d} \le K < b e^{-d}$

$$
C(K) = q \cdot \frac{(b_{-d} - K)^2}{2 \Delta_{-d}}
+ p \left( \frac{a_0 + b_0}{2} - K \right)
+ (1 - p - q) \left( \frac{a_d + b_d}{2} - K \right)
$$

### Case 3: $b e^{-d} \le K < a$

$$
C(K) = p \left( \frac{a_0 + b_0}{2} - K \right)
+ (1 - p - q) \left( \frac{a_d + b_d}{2} - K \right)
$$

### Case 4: $a \le K < b$

$$
C(K) = p \cdot \frac{(b_0 - K)^2}{2 \Delta_0}
+ (1 - p - q) \left( \frac{a_d + b_d}{2} - K \right)
$$

### Case 5: $b \le K < a e^d$

$$
C(K) = (1 - p - q) \left( \frac{a_d + b_d}{2} - K \right)
$$

### Case 6: $a e^d \le K < b e^d$

$$
C(K) = (1 - p - q) \cdot \frac{(b_d - K)^2}{2 \Delta_d}
$$

### Case 7: $K \ge b e^d$

$$
C(K) = 0
$$

Put cases follow symmetric logic using $\Phi_{\text{put}}$.


## 5. Third Equation: Forward Constraint

The forward price (risk-neutral expectation of $S_T$):

$$
F = \mathbb{E}[S_T] = q \cdot \mu e^{-d} + p \cdot \mu + (1 - p - q) \cdot \mu e^d,
\quad \text{where } \mu = \frac{a + b}{2}
$$


## 6. Calibration: Solve for $(p, q, d)$

Use three equations:

- Call price: $C_{\text{mkt}} = C(K=F)$
- Put price: $P_{\text{mkt}} = P(K=F)$
- Forward equation above

###  General $(p, q, d)$

Minimize objective:
$$
\text{loss} = (F_{\text{model}} - F)^2 + (C_{\text{model}} - C_{\text{mkt}})^2 + (P_{\text{model}} - P_{\text{mkt}})^2
$$

Use `scipy.optimize.minimize` with constraints $p \ge 0$, $q \ge 0$, $p + q \le 1$, $d > 0$


## Output

- Exact analytical expressions for call/put across all strikes
- Can be used to compute:
  - Implied volatility surface
  - 25Δ Risk Reversal and Butterfly
  - Greeks: Delta, Vega, etc.

In [2]:
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Define Φ_call and Φ_put for vectorized strike K and fixed x
def phi_call_vector(K, x):
    ax = a * np.exp(x)
    bx = b * np.exp(x)
    Dx = bx - ax
    result = np.piecewise(K,
                          [K <= ax, (K > ax) & (K < bx), K >= bx],
                          [lambda K: (ax + bx) / 2 - K,
                           lambda K: (bx - K) ** 2 / (2 * Dx),
                           0.0])
    return result

def phi_put_vector(K, x):
    ax = a * np.exp(x)
    bx = b * np.exp(x)
    Dx = bx - ax
    result = np.piecewise(K,
                          [K <= ax, (K > ax) & (K < bx), K >= bx],
                          [0.0,
                           lambda K: (K - ax) ** 2 / (2 * Dx),
                           lambda K: K - (ax + bx) / 2])
    return result

# Full 3-variable calibration: objective function
def calibration_objective(x):
    p, q, d = x
    if p < 0 or q < 0 or (p + q > 1) or d <= 0:
        return 1e6  # infeasible
    w0 = p
    w1 = q
    w2 = 1 - p - q
    f_model = mu * (w1 * np.exp(-d) + w0 + w2 * np.exp(d))
    c_model = w1 * phi_call(F, -d) + w0 * phi_call(F, 0) + w2 * phi_call(F, d)
    p_model = w1 * phi_put(F, -d) + w0 * phi_put(F, 0) + w2 * phi_put(F, d)
    return (f_model - F) ** 2 + (c_model - C_mkt) ** 2 + (p_model - P_mkt) ** 2

# Initial guess: p, q, d
x0 = [0.5, 0.25, 0.01]
bounds = [(0, 1), (0, 1), (0.001, 1.0)]

res_full = minimize(calibration_objective, x0=x0, bounds=bounds)
p_fit, q_fit, d_fit = res_full.x

# Evaluate PDF over K for plotting
K_vals = np.linspace(7.70, 7.90, 200)
pdf_vals = (
    q_fit / (b * np.exp(-d_fit) - a * np.exp(-d_fit)) * ((K_vals >= a * np.exp(-d_fit)) & (K_vals <= b * np.exp(-d_fit))) +
    p_fit / (b - a) * ((K_vals >= a) & (K_vals <= b)) +
    (1 - p_fit - q_fit) / (b * np.exp(d_fit) - a * np.exp(d_fit)) * ((K_vals >= a * np.exp(d_fit)) & (K_vals <= b * np.exp(d_fit)))
)

# Plot the calibrated PDF
plt.figure(figsize=(8, 4))
plt.plot(K_vals, pdf_vals, label="Calibrated Risk-Neutral PDF")
plt.axvline(F, color='gray', linestyle='--', label='Forward (ATM strike)')
plt.title("Risk-Neutral Density for USD/HKD with Jump Model")
plt.xlabel("Strike K")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

(p_fit, q_fit, d_fit)


NameError: name 'mu' is not defined

In [3]:
# Consolidate all required functions and run global calibration again

import numpy as np
from scipy.optimize import minimize

# Constants (based on prior session)
a, b = 7.75, 7.85
mu = 0.5 * (a + b)
F = mu  # Approximate ATM forward
C_mkt = 0.025  # Example market ATM call price
P_mkt = 0.020  # Example market ATM put price

# Define elementary phi functions
def phi_call(K, x):
    ax, bx = a * np.exp(x), b * np.exp(x)
    Dx = bx - ax
    if K <= ax:
        return (ax + bx) / 2 - K
    elif ax < K < bx:
        return (bx - K)**2 / (2 * Dx)
    else:
        return 0.0

def phi_put(K, x):
    ax, bx = a * np.exp(x), b * np.exp(x)
    Dx = bx - ax
    if K <= ax:
        return 0.0
    elif ax < K < bx:
        return (K - ax)**2 / (2 * Dx)
    else:
        return K - (ax + bx) / 2

# Call price by region
def call_price_7case_with_label(K, p, q, d):
    components = []
    for x, label in zip([-d, 0.0, d], ['-d', '0', '+d']):
        ax, bx = a * np.exp(x), b * np.exp(x)
        Dx = bx - ax
        if K <= ax:
            value, region = (ax + bx) / 2 - K, "linear"
        elif ax < K < bx:
            value, region = (bx - K)**2 / (2 * Dx), "quadratic"
        else:
            value, region = 0.0, "zero"
        weight = q if label == '-d' else p if label == '0' else 1 - p - q
        components.append({
            "jump": label,
            "support": (ax, bx),
            "type": region,
            "weight": weight,
            "contrib": weight * value
        })
    total_price = sum(c["contrib"] for c in components)
    return total_price, components

# Generate 7 sample Ks across regions
def generate_strike_examples(d):
    return [
        a * np.exp(-d) - 0.01,
        (a * np.exp(-d) + b * np.exp(-d)) / 2,
        b * np.exp(-d) + 0.001,
        (a + b) / 2,
        b + 0.001,
        (a * np.exp(d) + b * np.exp(d)) / 2,
        b * np.exp(d) + 0.01
    ]

# Global calibration objective across all Ks
def global_calibration_loss(x, Ks, market_prices):
    p, q, d = x
    if not (0 <= p <= 1 and 0 <= q <= 1 and 0 <= p + q <= 1 and d > 0):
        return 1e6
    total_loss = 0
    for K, market_price in zip(Ks, market_prices):
        model_price, _ = call_price_7case_with_label(K, p, q, d)
        total_loss += (model_price - market_price)**2
    return total_loss

# Step 1: use single point to get initial d estimate
initial_d = 0.0028
K_examples = generate_strike_examples(initial_d)

# Step 2: simulate model prices at these K (using previously calibrated d=0.00273)
p0, q0, d0 = 0.0, 0.0, 0.00273
market_prices = [call_price_7case_with_label(K, p0, q0, d0)[0] for K in K_examples]

# Step 3: perform global calibration
x0_global = [0.2, 0.2, 0.01]
bounds_global = [(0, 1), (0, 1), (1e-5, 0.5)]
global_result = minimize(global_calibration_loss, x0=x0_global,
                         bounds=bounds_global, args=(K_examples, market_prices))

# Final calibrated values
p_global, q_global, d_global = global_result.x
p_global, q_global, 1 - p_global - q_global, d_global


(0.05794801727223921, 0.0, 0.9420519827277608, 0.0028792238937540887)

In [11]:
import numpy as np
from scipy.optimize import minimize_scalar

# Constants for numerical example
S0 = 7.80        # spot price
F = 7.85        # forward price
K = 7.83         # strike = ATM
T = 0.25         # time to maturity (in years)
atm_vol = 0.01  # ATM volatility (annualized)
a, b = 7.75, 7.85  # peg interval
ratio = 0.5       # q = ratio * p

# Black-Scholes ATM Call price approximation (ATM strike ~ forward)
from scipy.stats import norm

def black_call(F, K, T, sigma):
    d1 = 0.5 * sigma * np.sqrt(T)
    return np.exp(-0) * (F * norm.cdf(d1) - K * norm.cdf(d1))

C_mkt = black_call(F, K, T, atm_vol)  # F = forward

# Log forward ratio
alpha = np.log(F / S0)

# Loss function to minimize
def model_call_price(p):
    if p <= 0 or p >= 1 or (1 - p * (1 + 2 * ratio)) <= 0:
        return 1e6  # infeasible p
    q = ratio * p
    w0 = p
    w1 = 1 - p - q
    d = alpha / (1 - p * (1 + 2 * ratio))
    ae_pos = a * np.exp(d)
    be_pos = b * np.exp(d)
    mid_term = (ae_pos + be_pos) / 2
    call = w0 * ((b - K)**2 / (2 * (b - a))) + w1 * (mid_term - K)
    return call

def loss(p):
    return (model_call_price(p) - C_mkt) ** 2

# Perform scalar minimization
result = minimize_scalar(loss, bounds=(1e-5, 0.99), method='bounded')

# Extract calibrated parameters
p_star = result.x
q_star = ratio * p_star
d_star = alpha / (1 - p_star * (1 + 2 * ratio))
w_star = 1 - p_star - q_star
C_model = model_call_price(p_star)

calibration_result = {
    "Calibrated p": p_star,
    "Calibrated q": q_star,
    "Jump up weight (1 - p - q)": w_star,
    "Calibrated d": d_star,
    "Model Price": C_model,
    "Market Price": C_mkt,
    "Loss": loss(p_star)
}

calibration_result


{'Calibrated p': 1.5205548268696003e-05,
 'Calibrated q': 7.602774134348002e-06,
 'Jump up weight (1 - p - q)': 0.9999771916775969,
 'Calibrated d': 0.006389992425447511,
 'Model Price': 0.020001099674414765,
 'Market Price': 0.010019947093241388,
 'Loss': 9.962340684866396e-05}